# STT for CSE

## Assignment 1, Lab 2

### Goal 1:
Choose ONE medium-to-large scale open-source repository for a real-world project to analyze.

The repositories [Meltano](https://github.com/meltano/meltano), [Nostalgia for Infinity (FreqTrade trading bot logic)](https://github.com/iterativv/nostalgiaforinfinity) and [Ivy](https://github.com/ivy-llc/ivy), seem suitable. This notebook works on the Nostalgia for Infinity repository.

### Goal 2

Define the selection criteria.

Some metrics that can be considered are primary coding language, GitHub star rating, number of forks, number of commits, number of contributors and number of people watching. Choose the ones from these which seem relevant.
The authors of MineCraft filtered by language, number of stars and then number of commits. Number of contributors might also be a useful metric in addition to these, might prove to be useful in reducing the number of repositories that match my criteria considering I need to select just one repository. The [SEART](https://seart-ghs.si.usi.ch/) platform is useful for this.

Criteria


1.   Primarily in Python
2.   At least 1200 stars on GitHub
3.   Between 12000 and 25000 commits
4.   Created in the past 5 years

This reduced the search to 20 repositories of which I chose 1 using subjective criteria.

### Goal 3

Identify bug-fixing commits from the repository.

For each bug-fixing commit, store the following information (in csv format).

Hash | Message | Hashes of parents | Is a merge commit? | List of modified files

In [ ]:
!pip install -U transformers

from git import Repo
import csv
import subprocess
import re
import torch

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mamiksik/CommitPredictorT5") # model for commit creation
model = AutoModelForSeq2SeqLM.from_pretrained("mamiksik/CommitPredictorT5")

tokenizer2 = AutoTokenizer.from_pretrained("microsoft/phi-3-mini-4k-instruct") # model for rectification
model2 = AutoModelForCausalLM.from_pretrained("microsoft/phi-3-mini-4k-instruct", torch_dtype=torch.float16, device_map="auto")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 55.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
!git clone https://github.com/iterativv/nostalgiaforinfinity
%cd repo

Cloning into 'nostalgiaforinfinity'...
remote: Enumerating objects: 54799, done.
remote: Counting objects: 100% (237/237), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 54799 (delta 222), reused 142 (delta 137), pack-reused 54562 (from 3)
Receiving objects: 100% (54799/54799), 223.04 MiB | 9.15 MiB/s, done.
Resolving deltas: 100% (36435/36435), done.
[Errno 2] No such file or directory: 'repo'
/content


In [ ]:
infi = Repo('/content/nostalgiaforinfinity')

In [ ]:
all_commits = list(infi.iter_commits())
print(all_commits[0])
print(len(all_commits))
print(type(all_commits[0]))

111b03fbdf1111a8bf2af06c98f54ff921f08d21
16836
<class 'git.objects.commit.Commit'>


This gives you a list of the hexsha codes of all commits. A commit hexsha is a unique identifier for a commit in a Git repository, represented as a 40-character hexadecimal string generated using the SHA-1 hash function. It allows developers to reference specific commits and track changes in the repository's history.

Identifying the commits that fix bugs

In [ ]:
bug_terms = ['bug', 'fix', 'patch', 'issue', 'resolve', 'crash', 'solve', 'regression', 'fall back', 'assertion', 'coverity', 'reproducible',
             'stack-wanted', 'steps-wanted', 'testcase', 'steps to reproduce', 'fail', 'npe', 'except', 'broken', 'differential testing',
             'crash', 'overflow', 'problem', 'avoid', 'workaround', 'stop', 'break', 'freez', 'hang', 'error', 'leak']

In [ ]:
bug_commits = []
with open("bug_fixing_commits.csv", 'w', newline = "") as f:
  writer = csv.writer(f)
  writer.writerow(['Hash', 'Message', 'Parent Hashes', 'Is Merge Commit', 'Modified Files'])
  for commit in all_commits:
    msg_low = (commit.message).lower()
    bug_fix = False
    for word in bug_terms:
      if msg_low.find(word)!=-1:
        bug_fix = True
        break
    if not bug_fix:
      continue
    bug_commits.append(commit)
    hash = commit.hexsha
    msg = commit.message.strip()
    parents = commit.parents
    if len(parents)>1:
      merge = 'Y'
    else:
      merge = 'N'
    files = list(commit.stats.files.keys())
    writer.writerow([hash, msg, parents, merge, files])

In [ ]:
from google.colab import files
files.download('bug_fixing_commits.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!git diff 02edacdc15b7b81690dc492d2ce210de46eacfd6

usage: git diff --no-index [<options>] <path> <path>

Diff output format options
    -p, --patch           generate patch
    -s, --no-patch        suppress diff output
    -u                    generate patch
    -U, --unified[=<n>]   generate diffs with <n> lines context
    -W, --function-context
                          generate diffs with <n> lines context
    --raw                 generate the diff in raw format
    --patch-with-raw      synonym for '-p --raw'
    --patch-with-stat     synonym for '-p --stat'
    --numstat             machine friendly --stat
    --shortstat           output only the last line of --stat
    -X, --dirstat[=<param1,param2>...]
                          output the distribution of relative amount of changes for each sub-directory
    --cumulative          synonym for --dirstat=cumulative
    --dirstat-by-file[=<param1,param2>...]
                          synonym for --dirstat=files,param1,param2...
    --check               warn if changes introduce

### Goal 4

For each modified file (in the previous step), store the following (as csv).
Hash Message Filename

Source
Code
(before)

Source
Code
(current)

Diff LLM Inference
(fix type)

Rectified
Message

In [ ]:
%cd nostalgiaforinfinity

/content/nostalgiaforinfinity


In [ ]:
!git diff --name-only 2f7ad9dc61ca186066ac118c7f6987c2ea57a3e4^ 2f7ad9dc61ca186066ac118c7f6987c2ea57a3e4
# this command gives the names of the files which have been changed

tests/backtests/helpers.py


In [ ]:
!git diff 2f7ad9dc61ca186066ac118c7f6987c2ea57a3e4^ 2f7ad9dc61ca186066ac118c7f6987c2ea57a3e4 -- tests/backtests/helpers.py
# function for getting difference in this particular file for this particular commit

diff --git a/tests/backtests/helpers.py b/tests/backtests/helpers.py
index 5a9cca85..85d8a088 100644
--- a/tests/backtests/helpers.py
+++ b/tests/backtests/helpers.py
@@ -269,7 +269,6 @@ class BacktestResults:
   def _set__stats_pct(self):
     return {
       "duration_avg": self.full_stats.duration_avg,
-      "profit_sum_pct": self.full_stats.profit_sum_pct,
       "profit_mean_pct": self.full_stats.profit_mean_pct,
       "profit_total_pct": self.full_stats.profit_total_pct,
       "max_drawdown": self.results.max_drawdown_account * 100,


In [ ]:
with open("bug_commits_rectified.csv", 'w', newline = "") as f:
  writer = csv.writer(f)
  writer.writerow(['Hash', 'Message', 'Filename', 'Source Code (before)', 'Source Code (current)', 'Diff', 'LLM Inference (fix type)', 'Rectified Message'])
  for commit in bug_commits:
    for file_path in commit.stats.files:
      try:
        before = infi.git.show(f"{commit.parents[0].hexsha}:{file_path}")
        after = infi.git.show(f"{commit.hexsha}:{file_path}")
        commit_hash = commit.hexsha
        diff_text = subprocess.check_output(
          ["git", "diff", f"{commit_hash}^", commit_hash, "--", file_path],
          text=True  # str op
        )
        hunks = re.split(r'\n@@ .* @@\n', diff_text)
        new_comms = []
        for hunk in hunks:
          # print("Hunk:\n", hunk)
          input_text = commit.message + "diff" + hunk
          input_ids = tokenizer.encode(input_text, return_tensors="pt")
          output_ids = model.generate(input_ids, max_length=150, num_beams=5, early_stopping=True)
          output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
          print("Output text: \n", output_text)
          new_comms.append(output_text)
        rectifier_ip = "Combine these messages to create a clear, non-repetitive and informative commit message. This is the file path: "+file_path+".\n"
        for new_comm in new_comms:
          rectifier_ip += new_comm + "\n"
        rectifier_ip += "This is the original commit message: "+commit.message
        input_ids_rectifier = tokenizer.encode(rectifier_ip, return_tensors="pt")
        output_ids_rectifier = model.generate(input_ids_rectifier, max_length=150, num_beams=5, early_stopping=True)
        output_text_rect = tokenizer.decode(output_ids_rectifier[0], skip_special_tokens=True)
        writer.writerow([commit_hash, commit.message, file_path, before, after, diff_text, "; ".join(new_comms), output_text_rect])
      except Exception as e:
        print(e)
        continue

Output text: 
 update the latest version of the branch
Output text: 
 add more info to run_bot.py
Output text: 
 update backtests.py
Output text: 
 fix comment_ci_results
Output text: 
 add missing fix
Output text: 
 update nfixes
Output text: 
 update backtests.py
Output text: 
 fix comment_ci_results
Output text: 
 add nostalgiaforinfinityx6
Output text: 
 update backtests.yml
Cmd('git') failed due to: exit code(128)
  cmdline: git show 02edacdc15b7b81690dc492d2ce210de46eacfd6:tools/updatewithforce
  stderr: 'fatal: path 'tools/updatewithforce' exists on disk, but not in '02edacdc15b7b81690dc492d2ce210de46eacfd6''
Output text: 
 add missing file
Output text: 
 add jupu, jupu, jupu+, jupu+, jupu+ and jupu+
Output text: 
 add jup, ksm, ldo, link to static pair
Output text: 
 add jup, jupu, shib, virtUAL, wld
Output text: 
 add jup, jupu, jupu, jupu+, jupu+, jupu+, jupu+, jupu+
Output text: 
 add more pronouns
Output text: 
 add missing file
Output text: 
 add jup/jupu to static pair wh

In [ ]:
[commit_hash, commit.message, file_path, before, after, diff_text, "; ".join(new_comms), output_text_rect]

['86f13b602a7c03f575f4ce546e8bcbd3a8fcd1ba',
 "Merge branch 'jerome-benoit-fixes_update_nfx_docker_compose.sh'\n",
 'tools/update_nfx_docker_compose.sh',
 '#!/bin/sh\n\n### Prereqs:\n## 1. A directory with the NFIX repo setup on the main branch to run via docker compose\n## 2. Configure the localization from repo\n## OPTIONAL: configure token and chatId on .env\n\nNFI_PATH=\nENV_PATH=$NFI_PATH\nFREQTRADE_IMAGE_UPDATE=false\nFREQTRADE_IMAGE="freqtradeorg/freqtrade:stable"\nTELEGRAM_NOTIFICATION=true\n\n### Simple script that does the following:\n## 1. Pull NFIX repo\n## 2. Compare if have new commit\n## 3. Send message to telegram if .env is configured\n## 4. Stop and Start freqtrader via docker compose\n######################################################\n\n## FUNCTIONS\nload_env() {\n    local env_file="${1:-.env}"\n    if [ -f "$env_file" ]; then\n       export $(grep -v \'^#\' .env | xargs)\n    else\n        echo "$env_file not found"\n        exit 1\n    fi\n}\n\nsend_telegram_

### Goal 5

Rectifying messages (if needed) on a per file basis to better contextualize, and analyze fixes.

In [ ]:
pip install transformers accelerate torch

### Goal 6

(f) Evaluation: Research Questions1
:

• RQ1 (Developer eval.): Do developers use a precise commit message in the
fixing commit? Quantify the hit rate.
• RQ2 (LLM eval.): Does the LLM generate a precise commit message in the
fixing commit? Quantify the hit rate.
• RQ3 (Rectifier eval.): To what extent were you able to rectify the message?
Quantify the hit rate.